# Introduction to Seaborn

`Seaborn` is a library for creating data visualizations. More specifically, it is commonly used to create statistical graphs. It is built on top of `matplotlib` but offers a very distinct interface that is closely integrated with `pandas` data structures.

By storing your in a `pandas` `DataFrame` and wrangling it into the right shape, `seaborn` lets you create complex visualizations with only a single line of code.

This notebook follows and expands upon the main points of the [official `seaborn` tutorial](https://seaborn.pydata.org/tutorial.html):

First, we introduce `pandas` and the data structure that `seaborn`'s plotting functions expect. Then, we give an overview of the different categories of plotting functions that `seaborn` offers. Finally, we will produce a variety of example visualizations and discuss how to customize their annotations.

<div class="alert alert-block alert-info">

We will cover a lot of ground and introduce a large number of different functions. All of these functions have quite a few parameters that let you change their behavior. Instead of diving deep into all of these options, we suggest you check out each function's [documentation](https://seaborn.pydata.org/api.html) and the official [gallery of examples](https://seaborn.pydata.org/examples/index.html). There is a _lot_ to be explored in `seaborn` and this workhops can only scratch the surface!

</div>


## Data wrangling

`Seaborn` works best with data that is stored in tabular form in a `pandas` `DataFrame`. Since we want to focus on `seaborn` and data visualization in this session, we do not have time to discuss in-depth `DataFrame`s or the data wrangling capabilities of `pandas`. If you are interested in learning more about that, however, check out our [workshop on data wrangling using `pandas`](https://git.dartmouth.edu/lib-digital-strategies/RDS/workshops/data-science/dirty-data).

For the purposes of this session, we will zoom in on `pandas`' main function how to change the general structure of the data: `pivot`, `melt`, and `wide_to_long`.

To demonstrate these functions, we load two different datasets in two different shapes:


In [ ]:
import pandas as pd

penguins_long = pd.read_csv("../data/penguins_long.csv")
flights_wide = pd.read_csv("../data/flights_wide.csv")

### Long-form versus wide-form data

Let's take a look at the two different datasets we loaded:


In [ ]:
penguins_long

In [ ]:
flights_wide

We notice that they have a very different structure:

In `penguins_wide`, we have one variable per column and one observation per row. In `flights_wide`, we have one row per level of one the variables (`year`), and one column per level of the other variable (the month). Each cell contains a value that is related to the specific levels of the variables shown across the columns and rows (i.e., the value in a particular year and a particular month).

<figure>
<img src="https://seaborn.pydata.org/_images/data_structure_19_0.png" style="width:60%">
<figcaption align="left">Long-form versus wide-form data. Each color denotes a variable.</figcaption>
</figure>

We often see the wide format in spreadsheets with few variables: It is intuitive to a human reader to find the intersection of the levels of interest of the two variables and then read the value. However, if we want to process the dataset computationally, this format has two major disadvantages:

- The meaning of the data is unclear because there is no label that clearly states what the numbers represent
- The format does not scale well to more than three variables: Each new variable would require repeating all levels of the variable expressed in the columns

The long format may seem at first glance much harder to read, but it does not suffer from the above issues: Every variable is clearly labeled by its column header, and adding a variable simply means adding another column.

For computational data processing, we therefore generally prefer the long format, also called [tidy data](https://vita.had.co.nz/papers/tidy-data.pdf).


### How to get your data tidy

Because the wide format is often considered more human-readable, we frequently find wide data tables _in the wild_. Fortunately, `pandas` comes with a set of functions to restructure a `DataFrame`: `melt`, `pivot`, and `wide_to_long`.

The `melt` method lets you move a variable, whose levels are expressed as different columns, to a single column that contains one row per level.

<div class="alert alert-block alert-info">

The name _melt_ probably derives from the metaphor of thinking of the data table as a chunk of ice that you expose to heat on the right: The table will start to "melt" from the right, losing columns that trickle down and pool at the bottom, making the table narrower, and longer.

</div>

Let's try it with the flights data:


In [ ]:
flights_long = flights_wide.melt(
    id_vars="year",  # Which column should be used as the identifier variable (not melted)
    # All other columns will be melted!
    var_name="month",  # Name for the new variable/column
    value_name="n_passengers",  # Name for the column holding the values
)
flights_long

The inverse of the `melt` operation is `pivot`:


In [ ]:
flights_long.pivot(
    index="year",  # Which column to use as the index (each level of this variable will be one row)
    columns="month",  # Create a new column for each level of this variable
)

Finally, the function [`wide_to_long`](https://pandas.pydata.org/docs/reference/api/pandas.wide_to_long.html) can help you restructure more complex wide-form data with more than three variables. This is beyond the scope of this session, however.

<div class="alert alert-block alert-info">

You actually _can_ use wide-form data to create visualizations in `seaborn`. However, it is strongly encouraged to use long-form data to make the most out of `seaborn`'s ability to structure and label visualizations according to the variables in your data structure.


## The two categories of `seaborn` plotting functions

There is a great variety of plotting functions available within `seaborn`, but all of them fall into one of two categories: _axes-level_ functions or _figure-level_ functions.

An axes-level function is very similar to a `matplotlib` plotting function: It creates and returns a self-contained, single-axes plot.

A figure-level function, on the other hand, lets you create multiple axes with just a single line of code. We will see some examples of this in a little bit.

Figure-level functions also offer a single interface to multiple different axes-level functions:

<figure>
<img src="https://seaborn.pydata.org/_images/function_overview_8_0.png" style="width:60%">
<figcaption align="left">Organization of figure- and axes-level functions.</figcaption>
</figure>

In the next couple of sections, we will see how to use both kinds of functions.


## Visualizing statistical relationships

Two kinds of charts can be used to show the relationship between two or more variables: line plots and scatter plots.

Line plots work best when you have clear, continuous independent and dependent variables. The flights data, for example, is a time series and therefore fits this description.

You can create a line plot with `seaborn` by using the function [`lineplot`](https://seaborn.pydata.org/generated/seaborn.lineplot.html), passing it the data, and specifying the name of the variable to be shown on the _x_ and _y_ axis, respectively:


In [ ]:
import seaborn as sns

sns.lineplot(flights_long.query("year == 1950"), x="month", y="n_passengers")

In the above example, we only showed the series of a single year. If we keep the data from all years, there are now multiple values for `n_passengers` that we want to plot for each value of `month`. In this case, when there are several _y_ values for the same value of _x_, `seaborn` shows the mean of these values and the confidence interval:


In [ ]:
sns.lineplot(flights_long, x="month", y="n_passengers")

If instead we would like to see a different line per year, we can use the `style` paramter. If we specify a variable as the argument, the data will be grouped by that variable and each group will be plotted with a different line style:


In [ ]:
sns.lineplot(flights_long, x="month", y="n_passengers", style="year")

If the grouping variable has more than a few levels, this can be hard to process for the human eye, however. We could therefore also change the color of each line using the `hue` parameter:


In [ ]:
sns.lineplot(flights_long, x="month", y="n_passengers", style="year", hue="year")

The penguins dataset is a good example for a collection of observations that do not have a sequential relationship. We could therefore visualize some of the variables using [`scatterplot`](https://seaborn.pydata.org/generated/seaborn.scatterplot.html):


In [ ]:
sns.scatterplot(penguins_long, x="bill_length_mm", y="flipper_length_mm")

**Exercise:**

1. Create a scatter plot of the `bill_length_mm` and the `flipper_length_mm`, where the color and marker style is different for each `species`. **Hint:** This works just like it did in `lineplot`!


Since the interface of these functions is so similar, the figure-level function [`relplot`](https://seaborn.pydata.org/generated/seaborn.relplot.html) can be used to create either of them:


In [ ]:
sns.relplot(
    penguins_long,
    kind="scatter",
    x="bill_length_mm",
    y="flipper_length_mm",
    hue="species",
    style="species",
)

The main advantage of `relplot` is that, as a figure-level function, we can now create multiple axes based on yet another grouping variable (so-called _facets_):


In [ ]:
sns.relplot(
    penguins_long,
    kind="scatter",
    x="bill_length_mm",
    y="flipper_length_mm",
    col="island",
)

**Exercise:**

1. Use `relplot` to visualize a scatter plot of the `bill_length_mm` and the `body_mass_g`, with one row per `species`, one column per `island`, and a different `hue` for each `sex`.


## Visualizing distributions of data

If we are intereted in visualizing the univariate distribution of a variable, we have a few options to choose from: a histogram, a probability density curve, a cumulative distribution function, or a marginal distribution. Just like with the relational plots, all of these functions are also accessible through a figure-level function.

Let's walk through them one by one, using the penguins as an example!

The histogram organizing the data by grouping them into ranges (a.k.a. _binning_) and then computes some sort of statistic on it, e.g. the count.

For example, we can quickly familiarize ourselves with the body mass of our penguins using the default settings of the function [`histplot`](https://seaborn.pydata.org/generated/seaborn.histplot.html):


In [ ]:
sns.histplot(penguins_long, x="body_mass_g")

By default, the calculated statistic in each bin is the number of observations that fall into it. We can change this by supplying and argument to the `stat` parameter. To get the percentage of the total data that fall into each range, we can for example do this:


In [ ]:
sns.histplot(penguins_long, x="body_mass_g", stat="percent")

We can also change the width of each bin using the corresponding parameter:


In [ ]:
sns.histplot(penguins_long, x="body_mass_g", stat="percent", binwidth=250)

**Exercise:**

- Plot the histogram in different colors for each `sex`! **Hint:** `histplot` uses many of the same parameters as `scatterplot`!
- Add a `y` variable of you choice. What do you see for a categorical `y`, and what for a continuous `y`?


If the sample size is sufficiently large, we can also estimate its probablity density function by using the function [`kdeplot`](https://seaborn.pydata.org/generated/seaborn.kdeplot.html), which stands for _kernel density estimation_ and is essentially a smoothed histogram:


In [ ]:
sns.kdeplot(penguins_long, x="body_mass_g")

**Exercise:**

1. Once again, use different colors for each level of the variable `sex`! What do you observe compared to using `histplot`?
2. Change the degree of smoothing by changing the parameter `bw_adjust` to different values in the range from `0.1` to `1.0`. What do you observere for very large and very low values of `bw_adjust`?


Another way to look at your data's distribution is by calculating the empirical cumulative distribution using [`ecdfplot`](https://seaborn.pydata.org/generated/seaborn.ecdfplot.htm). This visualization is great if you want be able to show the perentiles in your data:


In [ ]:
sns.ecdfplot(penguins_long, x="body_mass_g")

Sometimes we want to see the actual datapoints in addition to the computed statistics. For this case, we can add a marginal distribution to another plot using [`rugplot`](https://seaborn.pydata.org/generated/seaborn.rugplot.html):


In [ ]:
sns.kdeplot(penguins_long, x="body_mass_g")
sns.rugplot(penguins_long, x="body_mass_g")

We can even do this in combination with plots that are of a different "family", e.g. `scatterplot`:


In [ ]:
sns.scatterplot(
    penguins_long,
    x="bill_length_mm",
    y="flipper_length_mm",
)
sns.rugplot(
    penguins_long,
    x="bill_length_mm",
    y="flipper_length_mm",
)

Similar to the relational plots, we can use a figure-level function to easily switch between these different kinds of distributionplots. In this case, that function is called [`displot`](https://seaborn.pydata.org/generated/seaborn.displot.html):


In [ ]:
sns.displot(penguins_long, kind="kde", x="bill_length_mm")

**Exercise:**

1. Plot the probability density estimations (`"kde"`) of the `body_mass_g` of the penguins. Create one axes per `species`. In each axes, plot each `sex` in a different color.


## Visualizing categorical data

If you want to show the relationship between a numerical and a categorical variable, there are a number of functions to choose from (see the figure above). In this notebook, we are going to highlight four of them: the count plot, the bar chart, the categorical scatter plot, and the box-and-whisker plot.

Each of them is essentially the categorical equivalent of one of the plots we have discussed above. The count plot, for example, is in a sense a histogram of a categorical variable: How many observations do we have per category. In `seaborn`, we can create such a chart using [`countplot`](https://seaborn.pydata.org/generated/seaborn.countplot.html):


In [ ]:
sns.countplot(penguins_long, x="island")

**Exercise:**

1. Show a different bar per species


If we don't wish to just count the number of observations, but instead want to calculate some kind of statistic on all observations within a category, we can create a bar chart using the [`barplot`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function:


In [ ]:
sns.barplot(flights_long, x="year", y="n_passengers")

By default, `barplot` calculates the mean across all observations in each category and displays the 95 % confidence interval using errorbars. You can change this by using the parameters `estimator` and `errorbar`:


In [ ]:
sns.barplot(flights_long, x="year", y="n_passengers", estimator="sum", errorbar=None)

If we want to get a better view of the spread of the continuous variable, we can use a categorical scatter plot. The function to use here is called [`stripplot`](https://seaborn.pydata.org/generated/seaborn.stripplot.html):


In [ ]:
sns.stripplot(penguins_long, x="species", y="bill_length_mm")

To get a more quantitative representation of these distributions, we can use a box-and-whisker plot, where the box shows the quartiles and the whiskers extend to the rest of the distribution. It also uses a rule for outlier detection and marks such observations as circles. The function to use for this is called [`boxplot`](https://seaborn.pydata.org/generated/seaborn.boxplot.html):


In [ ]:
sns.boxplot(penguins_long, x="species", y="body_mass_g", hue="sex")

The figure-level function for categorical plots is called `catplot` and can be used very much like `relplot` and `displot`:


In [ ]:
sns.catplot(
    penguins_long, x="species", y="bill_length_mm", hue="sex", col="island", kind="box"
)

## Creating multi-plot visualizations

While the figure-level functions are a great way to create plots showing multiple facets of our data, each created plot has to be of the same kind. Sometimes, however, we want to show different perspectives on the same data by using different plots for the same variable.

Here, we want to show two examples of `seaborn` plots that combine different views of our data: `jointplot` and `pairplot`.

The `jointplot` function combines a bivariate relationship plot of two variables with univariate distribution plots of each variable:


In [ ]:
sns.jointplot(penguins_long, x="bill_length_mm", y="flipper_length_mm", hue="species")

The `pairplot` function takes in a set of variables and plots a separate scatter plots for each pair of variables:


In [ ]:
sns.pairplot(penguins_long, hue="species")

## Annotating figures

While `seaborn` uses the information in the data frame to label the graphs in a helpful way, we can fine-tune them as needed.

Since `seaborn` is built on `matplotlib`, we need to get a handle of the `Axes` artist and can then use the same syntax as we would in pure `matplotlib`.

Axes-level functions return the `Axes` directly:


In [ ]:
ax = sns.boxplot(penguins_long, x="species", y="body_mass_g", hue="sex")

# The Axes object is a matplotlib object! You therefore have to check matplotlib's documentation to learn how to interact with it
ax.set_xlabel("Species")
ax.set_ylabel("Body mass [g]")

Figure-level functions are a bit more complex, since they return a `seaborn` object called `FacetGrid`, which manages the collection of axes that the function produces.

You could access the individual axes through its `axes` property, but `FacetGrid` conveniently defines a few methods that let us change some elements in all axes at the same time:


In [ ]:
g = sns.catplot(
    penguins_long, x="sex", y="bill_length_mm", hue="island", col="species", kind="box"
)

g.set_xlabels("Sex")
g.set_ylabels("Bill length [mm]")
g.set_titles("{col_name}")

## Next steps

As mentioned at the top of this notebook, `seaborn` has a lot to offer and this overview is only the first step of your journey.

To explore `seaborn` further, we recommend checking out [the official tutorial](https://seaborn.pydata.org/tutorial.html) and [the extensive example gallery](https://seaborn.pydata.org/examples/index.html).

One particular topic that we left out in this session is figure aesthetics and style. Since all figures created by `seaborn` consist of `matplotlib` Artists under-the-hood, you can use all the styling options that `matplotlib` offers. In addition, `seaborn` lets you define and use themes that apply styling to all visualizations in a session. You can choose from a collection of pre-defined styles or even create your own.

For more on themes and color styles, check out this [section in the official tutorial](https://seaborn.pydata.org/tutorial/aesthetics.html).


<table>
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>
